# Setup environment

In [1]:
# install openslide dependencies
!sudo apt-get install openslide-tools
!sudo apt-get install python-openslide
!pip install openslide-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openslide-tools amd64 3.4.1+dfsg-5build1 [13.8 kB]
Fetched 104 kB in 1s (72.1 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readl

In [2]:
# install flamingo and histogpt
!pip install flamingo-pytorch --no-deps
!pip install git+https://github.com/marrlab/HistoGPT

  Cloning https://github.com/marrlab/HistoGPT to /tmp/pip-req-build-4pebi0ej
  Running command git clone --filter=blob:none --quiet https://github.com/marrlab/HistoGPT /tmp/pip-req-build-4pebi0ej
  Resolved https://github.com/marrlab/HistoGPT to commit 41b7306efcf596e085d9c58c1e7ec39484c66558
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━

In [5]:
# check whether to use a gpu or cpu
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Simple examples

In [4]:
# make a forward pass through the model
from transformers import BioGptConfig
from histogpt.models import HistoGPTForCausalLM, PerceiverResamplerConfig

histogpt = HistoGPTForCausalLM(BioGptConfig(), PerceiverResamplerConfig())
histogpt = histogpt.to(device)

text = torch.randint(0, 42384, (1, 256)).to(device)
image = torch.rand(1, 1024, 768).to(device)

print(histogpt(text, image).logits.size())

torch.Size([1, 256, 42384])


In [42]:
# generate text autoregressively
from histogpt.helpers.inference import generate

output = generate(
    model=histogpt,
    prompt=torch.randint(0, 42384, (1, 2)),
    image=torch.rand(1, 2, 768),
    length=256,
    top_k=40,
    top_p=0.95,
    temp=0.7,
    device=device
)

print(output.size())

torch.Size([1, 79])


# Generate reports from features

In [43]:
# download model weights
!wget https://huggingface.co/marr-peng-lab/histogpt/resolve/main/histogpt-1b-6k-pruned.pth?download=true

--2024-03-15 13:05:35--  https://huggingface.co/marr-peng-lab/histogpt/resolve/main/histogpt-1b-6k-pruned.pth?download=true
Resolving huggingface.co (huggingface.co)... 13.35.7.57, 13.35.7.5, 13.35.7.38, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/f6/8f/f68faf0906e39e8c3590cdbdd523457dc01bcea2a52d9de48cd7b06821eaac6a/16835f1069ffcfb5b379f3d1423fbf3d99a679d1b426e7b28c4604c8e1cd6956?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27histogpt-1b-6k-pruned.pth%3B+filename%3D%22histogpt-1b-6k-pruned.pth%22%3B&Expires=1710767135&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDc2NzEzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y2LzhmL2Y2OGZhZjA5MDZlMzllOGMzNTkwY2RiZGQ1MjM0NTdkYzAxYmNlYTJhNTJkOWRlNDhjZDdiMDY4MjFlYWFjNmEvMTY4MzVmMTA2OWZmY2ZiNWIzNzlmM2QxNDIzZmJmM2Q5OWE

In [8]:
# download example features
!wget https://huggingface.co/marr-peng-lab/histogpt/resolve/main/2023-03-06%2023.51.44.h5?download=true

--2024-03-15 12:45:46--  https://huggingface.co/marr-peng-lab/histogpt/resolve/main/2023-03-06%2023.51.44.h5?download=true
Resolving huggingface.co (huggingface.co)... 13.35.7.38, 13.35.7.5, 13.35.7.57, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/f6/8f/f68faf0906e39e8c3590cdbdd523457dc01bcea2a52d9de48cd7b06821eaac6a/72aaa4f690facfa0b02ffcec2b327e480933e134faf8633307097273294f6b49?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%272023-03-06%252023.51.44.h5%3B+filename%3D%222023-03-06+23.51.44.h5%22%3B&Expires=1710765946&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDc2NTk0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y2LzhmL2Y2OGZhZjA5MDZlMzllOGMzNTkwY2RiZGQ1MjM0NTdkYzAxYmNlYTJhNTJkOWRlNDhjZDdiMDY4MjFlYWFjNmEvNzJhYWE0ZjY5MGZhY2ZhMGIwMmZmY2VjMmIzMjdlNDgwOTMzZT

In [13]:
# load model weights
PATH = '/content/histogpt-1b-6k-pruned.pth?download=true'
state_dict = torch.load(PATH, map_location=device)
histogpt.load_state_dict(state_dict, strict=True)

<All keys matched successfully>

In [35]:
# get text prompt and image features
import h5py
from transformers import BioGptTokenizer

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

prompt = 'Final diagnosis:'
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

with h5py.File('/content/2023-03-06 23.51.44.h5?download=true', 'r') as f:
    features = f['feats'][:]
    features = torch.tensor(features).unsqueeze(0).to(device)

In [40]:
# generate the pathology report
output = generate(
    model=histogpt,
    prompt=prompt,
    image=features,
    length=256,
    top_k=40,
    top_p=0.95,
    temp=0.7,
    device=device
)

decoded = tokenizer.decode(output[0, 1:])
print(decoded)

Final diagnosis: Basal cell carcinoma. Microscopic findings: In the epidermis, endophytic proliferations of basaloid cell strands are observed in solid nest and strand-shaped formations, featuring palisade-like core positions in the peripheral zones. There is a cellular inflammatory stroma reaction. Critical findings: The diagnosis is solid basal cell carcinoma with a tumor thickness of 0. 7 mm. The specimen is accurately and completely described.


# ToDo: Generate reports from images

In [ ]:
from histogpt.helpers.patching import get_models

ctranspath = get_models('cpu')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Load the large, most powerful model

In [44]:
# download model weights
!wget https://huggingface.co/marr-peng-lab/histogpt/resolve/main/histogpt-3b-6k-pruned.pth?download=true
!wget https://huggingface.co/marr-peng-lab/histogpt/resolve/main/biogpt-large-config.json?download=true

--2024-03-15 13:06:35--  https://huggingface.co/marr-peng-lab/histogpt/resolve/main/histogpt-3b-6k-pruned.pth?download=true
Resolving huggingface.co (huggingface.co)... 13.35.7.38, 13.35.7.57, 13.35.7.81, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/f6/8f/f68faf0906e39e8c3590cdbdd523457dc01bcea2a52d9de48cd7b06821eaac6a/6602576908ce06805c2b324ef15439f1e492246d8d241c4e5ef9ecf3f0292f65?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27histogpt-3b-6k-pruned.pth%3B+filename%3D%22histogpt-3b-6k-pruned.pth%22%3B&Expires=1710767195&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDc2NzE5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y2LzhmL2Y2OGZhZjA5MDZlMzllOGMzNTkwY2RiZGQ1MjM0NTdkYzAxYmNlYTJhNTJkOWRlNDhjZDdiMDY4MjFlYWFjNmEvNjYwMjU3NjkwOGNlMDY4MDVjMmIzMjRlZjE1NDM5ZjFlND

In [46]:
# load histogpt only if you have enough vram
import json

PATH = '/content/biogpt-large-config.json?download=true'
biogpt_config = BioGptConfig.from_pretrained(PATH)

PATH = '/content/histogpt-3b-6k-pruned.pth?download=true'
state_dict = torch.load(PATH, map_location=device)

histogpt = HistoGPTForCausalLM(biogpt_config, PerceiverResamplerConfig())
histogpt.load_state_dict(state_dict, strict=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 38.38 MiB is free. Process 14980 has 15.73 GiB memory in use. Of the allocated memory 14.87 GiB is allocated by PyTorch, and 504.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)